In [ ]:
def generate_responseV2(prompt):
    """Generate stateless response from model by using token-level slicing"""
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output_ids = model.generate(
        input_ids,
        max_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )[0]

    # Slice the output to only keep new tokens beyond input prompt
    generated_ids = output_ids[len(input_ids[0]):]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return response.strip()

In [ ]:
def router(user_query):

    reset_prompt = "\n\n--- NEW SESSION ---\n\n"
    few_shot = """
    Example:
    Query: "What is the ingredient list of The Ordinary Lactic Acid 5% + Ha?"
    Matching Phrase: "What is the ingredient list of"
    Category: "WEB_SEARCH"

    Query: "What is the ingredient list of Tata Harper Illuminating Moisturiser?"
    Matching Phrase: "What is the ingredient list of"
    Category: "WEB_SEARCH"

    Query: "What are customers saying about Olay Eye Cream - With Niacinamide & Pentapeptides on the Internet?"
    Matching Phrase: "What are customers saying about"
    Category: "CUSTOMER_REVIEWS"

    Query: "What are customers saying about Kay Beauty Matte HD Setting Loose Powder on the Internet?"
    Matching Phrase: "What are customers saying about"
    Category: "CUSTOMER_REVIEWS"

    Query: "What is the customer sentiment around Maybelline New York The Colossal Liner - Black on the Internet?"
    Matching Phrase: "What is the customer sentiment around"
    Category: "SENTIMENT_ANALYSIS"

    Query: "What is the customer sentiment around L'Oreal Revitalift Night Cream on the Internet?"
    Matching Phrase: "What is the customer sentiment around"
    Category: "SENTIMENT_ANALYSIS"

    Query: "Is zirconium-containing complexes in cosmetics safe?"
    Matching Phrase: None
    Category: "FACTUAL"

    Query: "Is Triclosan in over-the-counter cosmetics safe for me and the environment?"
    Matching Phrase: None
    Category: "FACTUAL"
    """
    """Orchestrator: Route query to appropriate channel"""
    router_prompt = f"""Classify this query into one of these categories *ONLY* based on the presence of these **exact phrases** (case-insensitive).
    Do not explain or answer anything else:
1. WEB_SEARCH
2. CUSTOMER_REVIEWS
3. FACTUAL
Classify as WEB_SEARCH only if the query contains the exact phrases: "What is the ingredient list of".
Classify as CUSTOMER_REVIEWS only if the query contains the exact phrases: "What are customers saying about".
Classify as SENTIMENT_ANALYSIS only if the query contains the exact phrases: "What is the customer sentiment around".
Any query NOT containing these exact phrases should be classified as FACTUAL.
Do not guess. Do not infer. Match literally.

Query: {user_query}

Category:"""
# I tried few router_prompts. They all mis-classify some queries incorrectly into WEB_SEARCH or FACTUAL.
# Above router_prompt had least mis-classifcations among all the router prompts I tried.

    response = generate_responseV2(reset_prompt + few_shot + router_prompt)
    if "WEB_SEARCH" in response.upper():
        return "WEB_SEARCH"
    elif "CUSTOMER_REVIEWS" in response.upper():
        return "CUSTOMER_REVIEWS"
    elif "SENTIMENT_ANALYSIS" in response.upper():
        return "SENTIMENT_ANALYSIS"
    elif "FACTUAL" in response.upper():
        return "FACTUAL"
    else:
        return "UNKNOWN"